Based on lab: [https://github.com/MicrosoftLearning/mslearn-azure-ml/blob/main/Labs/10/Log%20models%20with%20MLflow.ipynb]

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient


credential = DefaultAzureCredential()
# Check if given credential can get token successfully.
credential.get_token("https://management.azure.com/.default")


ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


In [3]:
%%writefile train-model-autolog.py

# import libraries
import mlflow
import argparse
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from mlflow.types.schema import Schema, ColSpec
from mlflow.models.signature import ModelSignature

def main(args):
    # enable autologging
    # mlflow.sklearn.autolog()

    # read data
    df = get_data(args.training_data)

    # split data
    X_train, X_test, y_train, y_test = split_data(df)

    # train model
    model = train_model(args.reg_rate, X_train, X_test, y_train, y_test)

    # evaluate model
    y_hat = eval_model(model, X_test, y_test)


    # create the signature manually
    input_schema = Schema([
        ColSpec("integer", "Pregnancies"),
        ColSpec("integer", "PlasmaGlucose"),
        ColSpec("integer", "DiastolicBloodPressure"),
        ColSpec("integer", "TricepsThickness"),
        ColSpec("integer", "DiastolicBloodPressure"),
        ColSpec("integer", "SerumInsulin"),
        ColSpec("double", "BMI"),
        ColSpec("double", "DiabetesPedigree"),
        ColSpec("integer", "Age"),
    ])

    output_schema = Schema([ColSpec("boolean")])

    # create the signature by inferring it from the datasets
    # signature = infer_signature(X_train, y_hat)

    signature = ModelSignature(inputs=input_schema, outputs=output_schema)

    # manually log the model
    mlflow.sklearn.log_model(model, "model", signature=signature)

# function that reads the data
def get_data(path):
    import os, glob, tempfile, shutil, csv
    from pathlib import Path

    p = Path(path)
    if p.is_dir():
        candidates = sorted(p.glob("*.csv"))
        if not candidates:
            raise FileNotFoundError(f"No CSV files in: {p}")
        p = candidates[0]

    # copy to local temp to avoid mount read(nbytes) issues
    fd, tmp = tempfile.mkstemp(suffix=p.suffix or ".csv"); os.close(fd)
    shutil.copyfile(str(p), tmp)

    # detect delimiter (fallback to comma)
    try:
        with open(tmp, "r", encoding="utf-8", errors="ignore") as f:
            sample = f.read(20000)
        sep = csv.Sniffer().sniff(sample, delimiters=[",",";","\t","|"]).delimiter
    except Exception:
        sep = ","

    # robust read with python engine
    try:
        df = pd.read_csv(tmp, engine="python", sep=sep)
    except UnicodeDecodeError:
        df = pd.read_csv(tmp, engine="python", sep=sep, encoding="latin-1")

    print(f"Preparing {len(df)} rows of data")
    return df

# function that splits the data
def split_data(df):
    print("Splitting data...")
    X, y = df[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness',
    'SerumInsulin','BMI','DiabetesPedigree','Age']].values, df['Diabetic'].values

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

    return X_train, X_test, y_train, y_test

# function that trains the model
def train_model(reg_rate, X_train, X_test, y_train, y_test):
    mlflow.log_param("Regularization rate", reg_rate)
    print("Training model...")
    model = LogisticRegression(C=1/reg_rate, solver="liblinear").fit(X_train, y_train)

    # mlflow.sklearn.save_model(model, args.model_output)

    return model

# function that evaluates the model
def eval_model(model, X_test, y_test):
    # calculate accuracy
    y_hat = model.predict(X_test)
    acc = np.average(y_hat == y_test)
    print('Accuracy:', acc)
    mlflow.log_metric("Accuracy", acc)

    # calculate AUC
    y_scores = model.predict_proba(X_test)
    auc = roc_auc_score(y_test,y_scores[:,1])
    print('AUC: ' + str(auc))
    mlflow.log_metric("AUC", auc)

    # plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_scores[:,1])
    fig = plt.figure(figsize=(6, 4))
    # Plot the diagonal 50% line
    plt.plot([0, 1], [0, 1], 'k--')
    # Plot the FPR and TPR achieved by our model
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.savefig("ROC-Curve.png")
    mlflow.log_artifact("ROC-Curve.png") 

    return y_hat   

def parse_args():
    # setup arg parser
    parser = argparse.ArgumentParser()

    # add arguments
    parser.add_argument("--training_data", dest='training_data',
                        type=str)
    parser.add_argument("--reg_rate", dest='reg_rate',
                        type=float, default=0.01)
    # parser.add_argument("--model_output", dest='model_output',
    #                     type=str)

    # parse args
    args = parser.parse_args()

    # return args
    return args

# run script
if __name__ == "__main__":
    # add space in logs
    print("\n\n")
    print("*" * 60)

    # parse args
    args = parse_args()

    # run main function
    main(args)

    # add space in logs
    print("*" * 60)
    print("\n\n")

Overwriting train-model-autolog.py


In [2]:
from azure.ai.ml import command

# configure job (Train Model)

job = command(
    code="./",
    command="python train-model-autolog.py --training_data diabetes-data/diabetes.csv",
    environment="AzureML-sklearn-1.1-ubuntu20.04-py38-cpu@latest",
    compute="ml-compute-ntb",
    display_name="diabetes-train-autolog",
    experiment_name="diabetes-training"
    )

returned_job = ml_client.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading boris.zaikin (6.18 MBs): 100

In [ ]:
envs = ml_client.environments.list()
for env in envs:
    print(env.name, env.version)

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Register The Model

job_name = returned_job.name
# job_name = "gentle_plate_bskd49s6gn"

run_model = Model(
    path=f"azureml://jobs/{job_name}/outputs/artifacts/paths/model/",
    name="mlflow-diabetes",
    description="Model created from run.",
    type=AssetTypes.MLFLOW_MODEL,
)
# Uncomment after adding required details above
ml_client.models.create_or_update(run_model)